In [ ]:
# librairies
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.metrics import roc_curve, auc, precision_score, precision_recall_curve
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import scipy.io as sp

In [ ]:
# 